In [1]:
import pandas as pa
from lxml import html
import requests

df = pa.read_csv("https://raw.githubusercontent.com/nurfnick/NIH_Blast_Scrape/main/Data.csv")

In [4]:
links = df['Links']

In [6]:
links[0]

'https://www.ncbi.nlm.nih.gov/nucleotide/LC544014.1?report=genbank&log$=nucltop&blast_rank=1&RID=SUH6767C016'

In [14]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
#driver = webdriver.Firefox(executable_path = 'C:\Program Files\Mozilla Firefox\geckodriver.exe')
driver = webdriver.Firefox()
driver.get(links[1])
time.sleep(5)
htmlSource = driver.page_source

In [15]:
htmlSource

'<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en"><head xmlns:xi="http://www.w3.org/2001/XInclude"><link rel="stylesheet" type="text/css" href="https://static.pubmed.gov/core/jig/1.15.3/css/jig.min.css"><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <!-- meta -->\n    <meta name="robots" content="index,nofollow,noarchive">\n<meta name="ncbi_app" content="entrez"><meta name="ncbi_db" content="nuccore"><meta name="ncbi_report" content="genbank"><meta name="ncbi_format" content="html"><meta name="ncbi_pagesize" content="20"><meta name="ncbi_sortorder" content="default"><meta name="ncbi_pageno" content="1"><meta name="ncbi_resultcount" content="1"><meta name="ncbi_op" content="retrieve"><meta name="ncbi_pdid" content="genbank"><meta name="ncbi_sessionid" content="CE892FCF18D788F1_0693SID"><meta name="ncbi_uidlist" content="2119754211"><meta name="ncbi_filter" content="all"><meta name="ncbi_stat" content="false"><meta name="ncbi_hitstat" content="

In [61]:
soup = BeautifulSoup(htmlSource, 'html.parser')
soup.find_all("span", {"class": "feature"})[0].find_all("a")[1].text

'27.0825 N 142.2097 E'

In [57]:
driver.get(links[42])
time.sleep(1)
htmlSource = driver.page_source
soup = BeautifulSoup(htmlSource, 'html.parser')
soup.find_all("span", {"class": "feature"})[0].text

'     source          1..658\n                     /organism="Aedes albopictus"\n                     /organelle="mitochondrion"\n                     /mol_type="genomic DNA"\n                     /isolate="M1010"\n                     /db_xref="taxon:7160"\n                     /sex="male"\n                     /country="China: Jiangsu province"\n                     /collection_date="Aug-2014"\n'

In [60]:
driver.get(links[16])
time.sleep(1)
htmlSource = driver.page_source
soup = BeautifulSoup(htmlSource, 'html.parser')
soup.find_all("span", {"class": "feature"})[0]

<span class="feature" id="feature_LC543988.1_source_0"><script type="text/javascript">if (typeof(oData) == "undefined") oData = []; oData.push ({gi:2119754161,acc:"LC543988",features: {}});if (!oData[oData.length - 1].features["source"]) oData[oData.length - 1].features["source"] = [];oData[oData.length - 1].features["source"].push([[1, 658]]);</script>     source          1..658
                     /organism="Aedes albopictus"
                     /organelle="mitochondrion"
                     /mol_type="genomic DNA"
                     /isolate="OG7"
                     /db_xref="taxon:<a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=7160">7160</a>"
                     /lat_lon="<a href="https://www.google.com/maps/place/27.0825+142.2097">27.0825 N 142.2097 E</a>"
                     /collected_by="Kimura,R."
                     /identified_by="Kimura,R._&amp;_Tsuda,Y."
</span>

In [48]:
text = soup.find_all("span", {"class": "feature"})[0].text

In [62]:
text[text.find("/lat_lon"):text.find("/collected_by")]

''

In [63]:
allTheText = []
listOfLats = []
driver = webdriver.Firefox()

for i in links:
    driver.get(i)
    time.sleep(1)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    try:
        text = soup.find_all("span", {"class": "feature"})[0].text
        allTheText.append(text)
        listOfLats.append(text[text.find("/lat_lon"):text.find("/collected_by")])
    except:
        driver.get(i)
        time.sleep(5)
        htmlSource = driver.page_source
        soup = BeautifulSoup(htmlSource, 'html.parser')
        text = soup.find_all("span", {"class": "feature"})[0].text
        allTheText.append(text)
        listOfLats.append(text[text.find("/lat_lon"):text.find("/collected_by")])        
        

In [65]:
len(listOfLats)

100

In [ ]:
#Really similar to the above but may go a little faster and generate a cleaner list of the lat/longs
allTheText = []
listOfLats = []
driver = webdriver.Firefox()

for i in links:
    driver.get(i)
    time.sleep(1)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    try:
        text = soup.find_all("span", {"class": "feature"})[0].text
    except:
        driver.get(i)
        time.sleep(5)
        htmlSource = driver.page_source
        soup = BeautifulSoup(htmlSource, 'html.parser')
        text = soup.find_all("span", {"class": "feature"})[0].text
    allTheText.append(text)
    try:
        listOfLats.append(soup.find_all("span", {"class": "feature"})[0].find_all("a")[1].text)
    except:
        listOfLats.append('')    

In [68]:
textfile = open("allTheText.txt", "w")

for element in allTheText:

    textfile.write(element + "\n")

textfile.close()

In [70]:
import pandas
df = pandas.DataFrame(data={"col1": listOfLats})
df.to_csv("LatLongs.csv", sep=',',index=False)